 # 0. 딥러닝 실습 환경 소개

In [1]:
import tensorflow as tf

### 1) 하이퍼파라미터 설정

In [2]:
EPOCHS = 5

### 2) 네트워크 구조 정의

In [3]:
#keras를 상속받아서 사용
class MyModel(tf.keras.Model):
    def __init__(self) : #init에서 앞으로 사용할 모든 layer를 정의
        super(MyModel, self).__init__()
        self.flatten = tf.keras.layers.Flatten(input_shape = (28, 28))
        self.d1 = tf.keras.layers.Dense(128, activation = 'sigmoid')
        self.d2 = tf.keras.layers.Dense(10, activation = 'softmax')
    
    def call(self, x) : #call에서 각 layer의 연결성을 구성
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

### 3) 학습loop 정의

In [23]:
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy) : #다양한 입력값을 받아서
    with tf.GradientTape() as tape : 
        predictions = model(images) #model에 input image을 넣어서 출력을 내고
        loss = loss_object(labels, predictions)#출력과 정답을 비교해서
    gradients = tape.gradient(loss, model.trainable_variables)        #loss, gradient를 점차 개선
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)



### 4) test loop 정의 : 학습 loop 과 거의 유사한데, gradient를 만드는 부분만 빠져있는 것이다.

In [21]:
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images)
    
    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

### 5) 실질적인 학습 : 학습 데이터셋 가져오고, 정리

In [13]:
#import and organize dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0

x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

### 6) 네트워크 생성

In [14]:
model = MyModel()

### 7) 손실함수, 최적화 알고리즘 정의

In [15]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

### 8)알고리즘 평가지표 설정 

In [16]:
train_loss = tf.keras.metrics.Mean(name = 'train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')

test_loss = tf.keras.metrics.Mean(name = 'test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'test_accuracy')





### 9) 알고리즘 학습, 테스트 루프

In [24]:
for epoch in range(EPOCHS) : 
    for images, labels in train_ds : 
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)
        
        for test_images, test_labels in test_ds : 
            test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)
            
        template = 'Epoch : {}, Loss : {}, Accuracy : {}, Test Loss : {}, Test Accuracy : {}'
        print(template.format(epoch + 1, 
                             train_loss.result(), 
                             train_accuracy.result() *100, 
                             test_loss.result(), 
                             test_accuracy.result() *100))

Epoch : 1, Loss : 2.5846405029296875, Accuracy : 3.125, Test Loss : 2.459831714630127, Test Accuracy : 10.40999984741211
Epoch : 1, Loss : 2.580331802368164, Accuracy : 7.8125, Test Loss : 2.4214465618133545, Test Accuracy : 11.045000076293945
Epoch : 1, Loss : 2.502699851989746, Accuracy : 10.416666030883789, Test Loss : 2.3896024227142334, Test Accuracy : 12.373332977294922
Epoch : 1, Loss : 2.486110210418701, Accuracy : 10.9375, Test Loss : 2.3609459400177, Test Accuracy : 14.255001068115234
Epoch : 1, Loss : 2.4454658031463623, Accuracy : 11.875, Test Loss : 2.3347837924957275, Test Accuracy : 16.507999420166016
Epoch : 1, Loss : 2.4160077571868896, Accuracy : 13.020833015441895, Test Loss : 2.310385227203369, Test Accuracy : 18.84833335876465
Epoch : 1, Loss : 2.3808157444000244, Accuracy : 15.625, Test Loss : 2.2873430252075195, Test Accuracy : 20.928571701049805
Epoch : 1, Loss : 2.341986656188965, Accuracy : 19.53125, Test Loss : 2.2657792568206787, Test Accuracy : 22.700000762

Epoch : 1, Loss : 1.6255956888198853, Accuracy : 62.5, Test Loss : 1.5961331129074097, Test Accuracy : 63.549530029296875
Epoch : 1, Loss : 1.6151434183120728, Accuracy : 62.88461685180664, Test Loss : 1.5885708332061768, Test Accuracy : 63.741539001464844
Epoch : 1, Loss : 1.6099001169204712, Accuracy : 63.068180084228516, Test Loss : 1.5810949802398682, Test Accuracy : 63.92772674560547
Epoch : 1, Loss : 1.6003608703613281, Accuracy : 63.47947692871094, Test Loss : 1.573696494102478, Test Accuracy : 64.11537170410156
Epoch : 1, Loss : 1.5937288999557495, Accuracy : 63.69485092163086, Test Loss : 1.5663671493530273, Test Accuracy : 64.3016128540039
Epoch : 1, Loss : 1.5882067680358887, Accuracy : 63.949275970458984, Test Loss : 1.5591049194335938, Test Accuracy : 64.48869323730469
Epoch : 1, Loss : 1.5801492929458618, Accuracy : 64.2410659790039, Test Loss : 1.5519124269485474, Test Accuracy : 64.67828369140625
Epoch : 1, Loss : 1.5728728771209717, Accuracy : 64.4366226196289, Test Lo

Epoch : 1, Loss : 1.2755465507507324, Accuracy : 71.77579498291016, Test Loss : 1.2575902938842773, Test Accuracy : 72.04912567138672
Epoch : 1, Loss : 1.271010398864746, Accuracy : 71.97342681884766, Test Loss : 1.2535995244979858, Test Accuracy : 72.15228271484375
Epoch : 1, Loss : 1.2676783800125122, Accuracy : 72.0703125, Test Loss : 1.2496527433395386, Test Accuracy : 72.2530517578125
Epoch : 1, Loss : 1.264015555381775, Accuracy : 72.16569519042969, Test Loss : 1.2457537651062012, Test Accuracy : 72.35116577148438
Epoch : 1, Loss : 1.2599217891693115, Accuracy : 72.30769348144531, Test Loss : 1.2419049739837646, Test Accuracy : 72.44507598876953
Epoch : 1, Loss : 1.2550866603851318, Accuracy : 72.42366790771484, Test Loss : 1.2381045818328857, Test Accuracy : 72.53404998779297
Epoch : 1, Loss : 1.2522692680358887, Accuracy : 72.51420593261719, Test Loss : 1.2343491315841675, Test Accuracy : 72.6198501586914
Epoch : 1, Loss : 1.2494348287582397, Accuracy : 72.57988739013672, Test 

Epoch : 1, Loss : 1.08722984790802, Accuracy : 75.96409606933594, Test Loss : 1.0657718181610107, Test Accuracy : 76.35850524902344
Epoch : 1, Loss : 1.0849885940551758, Accuracy : 75.97553253173828, Test Loss : 1.063339352607727, Test Accuracy : 76.41529083251953
Epoch : 1, Loss : 1.0826938152313232, Accuracy : 76.01973724365234, Test Loss : 1.060917615890503, Test Accuracy : 76.47205352783203
Epoch : 1, Loss : 1.079245686531067, Accuracy : 76.12892150878906, Test Loss : 1.0585086345672607, Test Accuracy : 76.52864074707031
Epoch : 1, Loss : 1.076548457145691, Accuracy : 76.18815612792969, Test Loss : 1.0561113357543945, Test Accuracy : 76.58489227294922
Epoch : 1, Loss : 1.0736198425292969, Accuracy : 76.24676513671875, Test Loss : 1.053726077079773, Test Accuracy : 76.64087677001953
Epoch : 1, Loss : 1.0716160535812378, Accuracy : 76.30477142333984, Test Loss : 1.051354169845581, Test Accuracy : 76.6961898803711
Epoch : 1, Loss : 1.069701075553894, Accuracy : 76.33012390136719, Test

Epoch : 1, Loss : 0.9581702351570129, Accuracy : 78.51250457763672, Test Loss : 0.9403018951416016, Test Accuracy : 79.13191986083984
Epoch : 1, Loss : 0.9570902585983276, Accuracy : 78.52340698242188, Test Loss : 0.9386566281318665, Test Accuracy : 79.16438293457031
Epoch : 1, Loss : 0.9549645185470581, Accuracy : 78.55902862548828, Test Loss : 0.9370177388191223, Test Accuracy : 79.19721984863281
Epoch : 1, Loss : 0.9540104269981384, Accuracy : 78.59436798095703, Test Loss : 0.9353805780410767, Test Accuracy : 79.23023986816406
Epoch : 1, Loss : 0.9523934721946716, Accuracy : 78.64173126220703, Test Loss : 0.9337440729141235, Test Accuracy : 79.26386260986328
Epoch : 1, Loss : 0.9508183002471924, Accuracy : 78.66421508789062, Test Loss : 0.9321094155311584, Test Accuracy : 79.29753112792969
Epoch : 1, Loss : 0.9495634436607361, Accuracy : 78.67431640625, Test Loss : 0.9304774403572083, Test Accuracy : 79.33140563964844
Epoch : 1, Loss : 0.9483263492584229, Accuracy : 78.6843338012695

...

Epoch : 2, Loss : 0.3950161635875702, Accuracy : 89.80142974853516, Test Loss : 0.38032037019729614, Test Accuracy : 90.27558898925781
Epoch : 2, Loss : 0.3948703408241272, Accuracy : 89.80511474609375, Test Loss : 0.38023826479911804, Test Accuracy : 90.27754974365234
Epoch : 2, Loss : 0.39477741718292236, Accuracy : 89.80716705322266, Test Loss : 0.38015609979629517, Test Accuracy : 90.27944946289062
Epoch : 2, Loss : 0.3946932554244995, Accuracy : 89.81084442138672, Test Loss : 0.3800741136074066, Test Accuracy : 90.2813491821289
Epoch : 2, Loss : 0.3945712149143219, Accuracy : 89.8128890991211, Test Loss : 0.37999191880226135, Test Accuracy : 90.28328704833984
Epoch : 2, Loss : 0.3947146534919739, Accuracy : 89.81169128417969, Test Loss : 0.3799091875553131, Test Accuracy : 90.28521728515625
Epoch : 2, Loss : 0.3945721685886383, Accuracy : 89.81535339355469, Test Loss : 0.37982630729675293, Test Accuracy : 90.28717041015625
Epoch : 2, Loss : 0.39439716935157776, Accuracy : 89.82064

Epoch : 2, Loss : 0.3896687924861908, Accuracy : 89.94194793701172, Test Loss : 0.3754001259803772, Test Accuracy : 90.38597106933594
Epoch : 2, Loss : 0.3894975185394287, Accuracy : 89.947021484375, Test Loss : 0.37532562017440796, Test Accuracy : 90.3875732421875
Epoch : 2, Loss : 0.3893508315086365, Accuracy : 89.95052337646484, Test Loss : 0.3752516508102417, Test Accuracy : 90.38912963867188
Epoch : 2, Loss : 0.3893117904663086, Accuracy : 89.95085906982422, Test Loss : 0.3751779794692993, Test Accuracy : 90.3906478881836
Epoch : 2, Loss : 0.38916873931884766, Accuracy : 89.954345703125, Test Loss : 0.37510475516319275, Test Accuracy : 90.39215850830078
Epoch : 2, Loss : 0.38910892605781555, Accuracy : 89.95468139648438, Test Loss : 0.3750315010547638, Test Accuracy : 90.3936996459961
Epoch : 2, Loss : 0.3889927864074707, Accuracy : 89.95659637451172, Test Loss : 0.37495800852775574, Test Accuracy : 90.39523315429688
Epoch : 2, Loss : 0.3888871371746063, Accuracy : 89.958503723144

Epoch : 2, Loss : 0.3841788172721863, Accuracy : 90.06668853759766, Test Loss : 0.37077903747558594, Test Accuracy : 90.48719024658203
Epoch : 2, Loss : 0.3840566873550415, Accuracy : 90.07002258300781, Test Loss : 0.370703786611557, Test Accuracy : 90.48893737792969
Epoch : 2, Loss : 0.3839435875415802, Accuracy : 90.07182312011719, Test Loss : 0.3706284761428833, Test Accuracy : 90.49071502685547
Epoch : 2, Loss : 0.38387590646743774, Accuracy : 90.07362365722656, Test Loss : 0.37055331468582153, Test Accuracy : 90.49250793457031
Epoch : 2, Loss : 0.38382571935653687, Accuracy : 90.07389068603516, Test Loss : 0.3704783022403717, Test Accuracy : 90.49432373046875
Epoch : 2, Loss : 0.3837624788284302, Accuracy : 90.07568359375, Test Loss : 0.3704032897949219, Test Accuracy : 90.49609375
Epoch : 2, Loss : 0.3836577832698822, Accuracy : 90.0790023803711, Test Loss : 0.37032830715179443, Test Accuracy : 90.497802734375
Epoch : 2, Loss : 0.3835695683956146, Accuracy : 90.07926940917969, Te

Epoch : 2, Loss : 0.37981748580932617, Accuracy : 90.16329956054688, Test Loss : 0.36621183156967163, Test Accuracy : 90.59085845947266
Epoch : 2, Loss : 0.3797033429145813, Accuracy : 90.16648864746094, Test Loss : 0.36614006757736206, Test Accuracy : 90.59249877929688
Epoch : 2, Loss : 0.3796250522136688, Accuracy : 90.1681900024414, Test Loss : 0.3660683035850525, Test Accuracy : 90.59406280517578
Epoch : 2, Loss : 0.37947502732276917, Accuracy : 90.1728515625, Test Loss : 0.36599674820899963, Test Accuracy : 90.5955810546875
Epoch : 2, Loss : 0.37942054867744446, Accuracy : 90.17306518554688, Test Loss : 0.3659253418445587, Test Accuracy : 90.59715270996094
Epoch : 2, Loss : 0.37950068712234497, Accuracy : 90.17476654052734, Test Loss : 0.3658539652824402, Test Accuracy : 90.59883117675781
Epoch : 2, Loss : 0.37936821579933167, Accuracy : 90.1794204711914, Test Loss : 0.3657827079296112, Test Accuracy : 90.60039520263672
Epoch : 2, Loss : 0.3792344331741333, Accuracy : 90.184066772

DeadlineExceededError: tf_data_meta_optimizer exceeded deadline. [Op:OptimizeDataset]